In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import cv2
import numpy as np

# ==============================================================================
# 1. 설정 및 경로 정의
# ==============================================================================
# ⚠️ 중요: 모델 파일 경로를 확인하세요.
MODEL_PATH = 'resnet_saved_models/resnet50_best_tp_tn_40_20251028_150108.pth'
NUM_CLASSES = 2
IMG_SIZE = 224
# 클래스 인덱스 0과 1에 해당하는 이름 (레이블링에 맞게 설정)
CLASS_NAMES = ['0 (왼쪽)', '1 (오른쪽)'] 

# ImageNet 표준 정규화 값
NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD = [0.229, 0.224, 0.225]

# 디바이스 설정 (GPU 사용 최적화)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"✅ 사용 디바이스: {device}")


# ==============================================================================
# 2. 모델 로드 및 전처리 정의
# ==============================================================================

def load_classification_model(model_path):
    """모델 구조를 로드하고 저장된 가중치를 불러옵니다."""
    # ResNet-50 구조 로드
    model = models.resnet50(weights=None) 
    
    # 마지막 FC 레이어를 2개의 클래스에 맞게 수정 (학습 시와 동일)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, NUM_CLASSES) 

    # 가중치 불러오기
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        model = model.to(device)
        model.eval() # 모델을 평가 모드로 설정 (필수)
        print(f"✅ 모델 가중치 로드 성공: {model_path}")
        return model
    except Exception as e:
        print(f"❌ 모델 로드 오류: {e}")
        return None

# 추론 시 사용할 전처리 파이프라인
inference_transform = transforms.Compose([
    transforms.ToPILImage(),             # OpenCV 배열을 PIL 이미지로 변환
    transforms.Resize(256),              # 크기 조정
    transforms.CenterCrop(IMG_SIZE),     # 중앙 크롭
    transforms.ToTensor(),               # Tensor로 변환
    transforms.Normalize(NORM_MEAN, NORM_STD) # 정규화
])


# ==============================================================================
# 3. 실시간 추론 루프
# ==============================================================================
# 모델 로드
model = load_classification_model(MODEL_PATH)

if model is None:
    exit()

# 카메라 초기화 (일반적으로 1번 웹캠)
cap = cv2.VideoCapture(1) # ⚠️ 카메라 인덱스를 0 또는 1 등으로 조정해야 할 수 있습니다.

if not cap.isOpened():
    print("❌ 오류: 카메라를 열 수 없습니다.")
    exit()

print("✅ 실시간 ResNet-50 분류 시작... 'q' 키를 눌러 종료하세요.")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # 1. 전처리 (프레임을 Tensor로 변환 및 정규화)
    # OpenCV (BGR) 이미지를 RGB로 변환해야 PyTorch/ImageNet 표준에 맞음
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 2. 전처리 파이프라인 적용
    input_tensor = inference_transform(rgb_frame).unsqueeze(0).to(device) # unsqueeze(0)로 배치 차원 추가

    # 3. 모델 추론 (torch.no_grad()는 추론 시 필수)
    with torch.no_grad():
        outputs = model(input_tensor)
        
    # 4. 결과 해석
    probabilities = torch.softmax(outputs, dim=1) # 확률로 변환
    conf, predicted_index = torch.max(probabilities, 1) # 최대 확률과 인덱스 추출
    
    predicted_class_name = CLASS_NAMES[predicted_index.item()]
    confidence = conf.item() * 100
    
    # 5. 화면에 출력
    label = f'Predicted: {predicted_class_name} ({confidence:.2f}%)'
    
    # 결과 텍스트를 화면 좌측 상단에 표시
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
    cv2.imshow("ResNet-50 Arrow Classifier (Press 'q' to quit)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
print("실시간 분류 종료.")